# 代码覆盖率
来源：[Code Coverage](https://www.fuzzingbook.org/html/Coverage.html)

In [ ]:
# 对使用cgi的url解码
def cgi_decode(s):
    # 数字找字母，下标法可能比较简单：https://blog.csdn.net/sinat_38816924/article/details/78440502
    # 字母找数字，字典是不错的选择
    hex_vaule = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    }

    exchange_value = {'+':' '}

    i = 0
    t = ''
    while i < len(s):
        # 如果是交换表中的内容直接替换
        # 如果是%的形式，计算转换
        # 其他情况直接赋值
        if s[i] in exchange_value.keys():
            t += exchange_value[s[i]]
            i = i+1
        elif s[i] == '%':
            high_value,low_value = s[i+1],s[i+2]
            if high_value in hex_vaule and low_value in hex_vaule:
                t += chr(hex_vaule[high_value]*16+hex_vaule[low_value])
                i += 3
            else:
                raise ValueError("Invalid encoding")
        else :
            t += s[i]
            i += 1
    
    return t

In [ ]:
# 测试下cig_decode程序
cgi_decode("Hello%2c+world%21")

## 黑盒测试
针对特殊用例。黑盒测试的优点是它可以发现指定行为中的错误。它独立于给定的实现，因此允许在实现之前创建测试。缺点是，实现的行为通常比指定的行为涵盖更多的领域，因此仅基于规范的测试通常不能涵盖所有的实现细节。

In [5]:
assert cgi_decode("a+b") == "a b"
assert cgi_decode("a%21b") == "a!b"
assert cgi_decode("abc") == "abc"

try:
    cgi_decode("%?")
    raise False
except:
    pass